
# Store-Specific Product Comparison Tool

1. Select a store and get the best-selling products.
2. Save the best-sellers to CSV.
3. Prompt the user to input a product name.
4. Search for that product across all stores and export results to CSV.
5. Let the user choose **which stores** to compare.
6. Display the **cheapest product** if available.


In [1]:

import pandas as pd
from fuzzywuzzy import process, fuzz
import matplotlib.pyplot as plt


# Helper to normalize item names
def clean_item_name(name):
    if isinstance(name, str):
        return name.lower().strip()
    return name

# Load all datasets
file_paths = {
    "IGA": "IGA 1.csv",
    "Foodland": "foodland_balaklava.csv",
    "Woolworths": "woolworths_cleaned.csv",
    "Coles": "coles_synthetic_dataset.csv"
    
}
dataframes = {k: pd.read_csv(v) for k, v in file_paths.items()}


In [2]:
# Ask the user to enter a product name for partial match searching
product_query = input("Enter product name to search: ").lower().strip()

# Function to convert a price column to numeric and drop rows with invalid/missing values
def clean_price_column(df, column):
    df[column] = pd.to_numeric(df[column], errors="coerce")  # Convert prices to numeric, coerce errors to NaN
    return df.dropna(subset=[column])  # Drop rows where the price is NaN

# Prepare a list to hold all matched rows from all stores
matched_rows = []

# Iterate through each store's DataFrame in the dictionary `dataframes`
for store, df in dataframes.items():
    # Dynamically detect column name for product and price across different formats
    name_col = "Item Name" if "Item Name" in df.columns else "Product Name"
    price_col = (
        "Item Price" if "Item Price" in df.columns else
        "Discounted Price" if "Discounted Price" in df.columns else
        "Best Price"
    )

    # Clean item name column if needed (could apply further normalization here)
    df[name_col] = df[name_col].apply(clean_item_name)  # Optional: a cleaning function you may define

    # Clean the price column (convert to numeric and drop invalids)
    df = clean_price_column(df, price_col)

    # Filter rows where the item name contains the search keyword (case-insensitive)
    matches = df[df[name_col].str.contains(product_query, na=False)].copy()

    # Add store name to each matching row
    matches["Store"] = store

    # Keep only relevant columns and rename them uniformly
    matches = matches[[name_col, price_col, "Store"]]
    matches.columns = ["Product Name", "Price", "Store"]

    # Append this store's matches to the results list
    matched_rows.append(matches)

# Combine matches from all stores into one DataFrame
matched_df = pd.concat(matched_rows, ignore_index=True)

# Save the final matched results to a CSV file
matched_df.to_csv("matching_products.csv", index=False)

# Print a confirmation message and return the result
print("Matching products saved to matching_products.csv")
matched_df


Matching products saved to matching_products.csv


,Product Name,Price,Store
0,primo trios minions ham cheese & banana chips 47g,4.50,Woolworths
1,mission corn chips cheesy nachos 230g,4.00,Woolworths
2,cadbury baking milk chocolate chips value pack...,6.00,Woolworths
3,cadbury baking milk chocolate chips 200g,5.00,Woolworths
4,cadbury baking dark chocolate chips 200g,5.00,Woolworths
...,...,...,...
2512,vege chips natural | 100g,2.45,Coles
2513,vege chips rice crackers sour cream & chives |...,2.45,Coles
2514,vege chips rice crackers sweet chilli & sour c...,2.45,Coles
2515,vege chips sea salt & vinegar | 100g,2.45,Coles


In [3]:

# Show all matched products
print("Available products for comparison:")
print(matched_df["Product Name"].unique())

# Ask which product to compare
compare_product = input("Enter the exact product name to compare: ").lower().strip()

# Ask which stores to include in comparison
available_stores = matched_df["Store"].unique()
print(f"Available stores: {', '.join(available_stores)}")
selected_stores = input("Enter comma-separated stores to compare (e.g. IGA,Foodland,Coles,Woolworths): ").split(",")

# Filter data
filtered = matched_df[
    (matched_df["Product Name"].str.lower() == compare_product) &
    (matched_df["Store"].isin([s.strip() for s in selected_stores]))
]

# Show cheapest or not found
if filtered.empty:
    print("The specified product was not found in the selected stores.")
else:
    cheapest = filtered.loc[filtered["Price"].idxmin()]
    print(f"Cheapest match found:Store: {cheapest['Store']}, Product: {cheapest['Product Name']}, Price: ${cheapest['Price']}")


Available products for comparison:
['primo trios minions ham cheese & banana chips 47g'
 'mission corn chips cheesy nachos 230g'
 'cadbury baking milk chocolate chips value pack 410g'
 'cadbury baking milk chocolate chips 200g'
 'cadbury baking dark chocolate chips 200g'
 'mission white strip corn chips 500g'
 'mission corn chips deli round 500g'
 'cadbury baking white chocolate chips 200g'
 'mission corn chips chilli & lime 230g' 'sonora corn chips salted 500g'
 'noshu 95% sugar free choc baking chips 140g'
 'noshu 97% sugar free dark choc baking chips 140g'
 'nong shim shrimp meat chip shrimp meat chips 75g'
 'orion turtle chips choco churros 160g'
 'cadbury cadbury caramilk chocolate baking chips 260g'
 'orion turtle chips corn soup 160g'
 'gravox best ever hot chip gravy pouch liquid gravy for chips 165g'
 'orion turtle chips flaming mala 160g'
 "green's chocolate chunk cookies with dark chocolate chips 400g"
 'orion turtle chips sweet vanilla 160g'
 'noshu sugar free white choc ba

In [4]:
# Generate top pricing matches across all stores for the searched product
# Reuse product_query from earlier step
store_results = []

for store, df in dataframes.items():
    name_col = "Item Name" if "Item Name" in df.columns else "Product Name"
    price_col = "Item Price" if "Item Price" in df.columns else "Discounted Price" if "Discounted Price" in df.columns else "Best Price"
    
    df[name_col] = df[name_col].apply(clean_item_name)
    df = clean_price_column(df, price_col)
    
    # Get top 5 cheapest matches that contain the product query
    matched = df[df[name_col].str.contains(product_query, na=False)].copy()
    matched["Store"] = store
    matched = matched.sort_values(price_col).head(5)
    matched = matched[[name_col, price_col, "Store"]]
    matched.columns = ["Product Name", "Price", "Store"]
    store_results.append(matched)

# Concatenate all top matched results
top_prices_df = pd.concat(store_results, ignore_index=True)
top_prices_df.to_csv("top_store_prices.csv", index=False)
print("Top matching store prices saved to top_store_prices.csv")
top_prices_df


Top matching store prices saved to top_store_prices.csv


,Product Name,Price,Store
0,red rock deli potato chips honey soy chicken d...,1.35,Woolworths
1,nong shim shrimp meat chip shrimp meat chips 75g,1.45,Woolworths
2,nong shim shrimp meat chip shrimp meat chips 75g,1.45,Woolworths
3,nong shim shrimp meat chip shrimp meat chips 75g,1.45,Woolworths
4,nong shim shrimp meat chip shrimp meat chips 75g,1.45,Woolworths
5,smiths grainwaves chips sour cream chives | 40g,1.50,Coles
6,smiths grainwaves chips sour cream chives | 40g,1.50,Coles
7,smiths grainwaves chips sour cream chives | 40g,1.50,Coles
8,smiths grainwaves chips sour cream chives | 40g,1.50,Coles
9,smiths grainwaves chips sour cream chives | 40g,1.50,Coles
